# Berkeley Earth data availability

Check data availability on global land for Berkeley Earth data. This notebook is used to determine what conditions to apply to the data.

In [ ]:
import copy

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from utils import berkeley, utils

**NOTE**: as of 12.03.2025: only data until 2024-08-31 -> don't analyse 2024

In [ ]:
CLIMATOLOGY_PERIOD = slice("1961", "1990")

end_year = "2023"

## Load data

In [ ]:
ds_tmax = berkeley.read("TMAX", time_period=slice(None, end_year))
ds_tmax = ds_tmax.load()

In [ ]:
# ds_tmin = berkeley.read("TMIN", time_period=slice(None, end_year))
# ds_tmin = ds_tmin.load()

## add the annual cycle

only for 1955 on (see below)

In [ ]:
# add climatology, so the annual max can be properly determined
ds_tmax_seas = berkeley.add_climatology(ds_tmax.sel(time=slice("1955", None)))

# ds_tmin_seas = berkeley.add_climatology(ds_tmin.sel(time=slice("1955", None)))

### calculate annual max/ min

In [ ]:
with xr.set_options(use_flox=True):
    txx = ds_tmax_seas.groupby("time.year").max("time", engine="flox")
    # tnn = ds_tmin_seas.groupby("time.year").min("time", engine="flox")

##  land mask

there are land grid points that never have any data -> remove those

In [ ]:
has_no_data_on_land_tmax = berkeley.land_without_any_data(ds_tmax)
# has_no_data_on_land_tmin = berkeley.land_without_any_data(ds_tmax)

# make sure min and max have the same missing data points
# assert (has_no_data_on_land_tmax == has_no_data_on_land_tmin).all()

f, ax = plt.subplots(1, 1)

has_no_data_on_land_tmax.plot(ax=ax)

### clean land mask

remove gridpoints without data

In [ ]:
land_mask = berkeley.clean_landmask(ds_tmax)

In [ ]:
land_mask.plot()

## Check data availability

#### calculate annual data availabiliy (fraction)

fraction of data available at each grid point for each year

In [ ]:
# get the fraction of valid data per year
notnull_tmax_map_ = berkeley.annual_data_availability(ds_tmax.temperature)
# notnull_tmin_map_ = berkeley.annual_data_availability(ds_tmin.temperature)

#### land-mean fraction of data for each year (w/o Antarctica)

In [ ]:
notnull_tmax = utils.land_mean(notnull_tmax_map_, land_mask)
# notnull_tmin = utils.land_mean(notnull_tmin_map_, land_mask)

In [ ]:
f, ax = plt.subplots(1, 1)

notnull_tmax.plot(ax=ax, label="T max")
# notnull_tmin.plot(ax=ax, label="T min")

ax.legend()

ax.set_ylabel("Land area available")

> Data avalability is generally good after 1950 and less good before. Restrict the rest of the analysis to after 1950. (Decide the exact year below).

In [ ]:
notnull_tmax_map = notnull_tmax_map_.sel(year=slice(1950, None))
# notnull_tmin_map = notnull_tmin_map_.sel(year=slice(1950, None))

### data availability requirements

e.g., require data for at least 90% of days

- 90% -> ~ 36 days
- 95% -> ~ 18 days
- 99% -> ~  4 days
can be missing

It is important that not too many days are missing - otherwise the annual min/ max can be wrong.

In [ ]:
days_required = [0, 0.90, 0.91, 0.92, 0.95, 0.99]  # % of data per year


def dta_av_required(data_notnull, *days_required):

    notnull = dict()

    for req in days_required:

        has_data = berkeley.require_valid(
            data_notnull, data_notnull, valid_days=req, valid_years=0
        )

        has_data = has_data.fillna(0)
        dta = utils.land_mean(has_data, land_mask)

        p = int(req * 100)
        notnull[f"{p:03d}"] = dta

    return notnull


# get annual data availability once we require at least x% of data each year

notnull_tmax_atleast = dta_av_required(notnull_tmax_map, *days_required)
# notnull_tmin_atleast = dta_av_required(notnull_tmin_map, *days_required)

In [ ]:
f, axs = plt.subplots(2, 1, layout="constrained", sharex=True)

ax = axs[0]

for key, value in notnull_tmax_atleast.items():
    value.plot(ax=ax, label=f"{int(key):d}%")

ax.set_xlabel("")
ax.set_title("Availability when requiring at least X% of data")

ax.set_ylabel("T max: Land area\navailable (%)")


# ax = axs[1]

# for key, value in notnull_tmin_atleast.items():
#     value.plot(ax=ax, label=f"{int(key):d}%")

# ax.set_ylabel("T min: Land area\navailable (%)")


for ax in axs:

    ax.legend()

    ax.axhline(0.95, color="0.1", lw=0.5)
    ax.axhline(1.00, color="0.1", lw=0.5)

    ax.axvline(1955, color="0.1", lw=1)

#### Conclusions

> - Requiring at least 90% or 91 % makes (almost) no difference
> - Requiring at least 92% or more makes (almost) no difference
> - After 1953 data availability is generally > 95%
> - But let's choose 1955 to have a nicer number

In [ ]:
notnull_tmax_map_1955 = notnull_tmax_map.sel(year=slice(1955, None))
# notnull_tmin_map_1955 = notnull_tmin_map.sel(year=slice(1955, None))

In [ ]:
def plot_data_av_map(data, variable, require):

    opt = {
        "transform": ccrs.PlateCarree(),
        "add_colorbar": False,
        "vmin": 0.50,
        "vmax": 1.0,
    }  # , "cmap": "Reds"}

    f, axs = plt.subplots(2, 1, subplot_kw={"projection": ccrs.PlateCarree()})

    axs = axs.flatten()

    ax = axs[0]
    h = data.where(land_mask).mean("year").plot(ax=ax, **opt)
    # notnull_tmin_map_1955

    ax.set_title(f"{variable}: Mean data availability")

    ax = axs[1]
    data = data.where(data >= require, 0).where(land_mask).mean("year")
    h = data.plot(ax=ax, **opt)

    ax.set_title(f"With valid_days >= {require}")

    plt.colorbar(h, ax=axs)

    for ax in axs:
        ax.coastlines()

In [ ]:
plot_data_av_map(notnull_tmax_map_1955, "T max", require=0.99)

In [ ]:
# plot_data_av_map(notnull_tmin_map_1955, "T min", require=0.99)

> There are three hotspots of missing data

## Compare data availability requirements

In [ ]:
txx_00_00 = berkeley.require_valid(
    txx, notnull_tmax_map, valid_days=0.0, valid_years=0.0
)
txx_99_00 = berkeley.require_valid(
    txx, notnull_tmax_map, valid_days=0.99, valid_years=0.0
)
txx_99_90 = berkeley.require_valid(
    txx, notnull_tmax_map, valid_days=0.99, valid_years=0.9
)

txx_00_00_land = utils.land_mean(txx_00_00, land_mask)
txx_99_00_land = utils.land_mean(txx_99_00, land_mask)
txx_99_90_land = utils.land_mean(txx_99_90, land_mask)

In [ ]:
txx_00_00_land.plot(label="0% of days; 0% of years")
txx_99_00_land.plot(label="99% of days; 0% of years")
txx_99_90_land.plot(label="99% of days; 90% of years")

plt.legend()

> Restricting the data makes almost no difference for land mean TXx.

In [ ]:
# tnn_00_00 = berkeley.require_valid(
#     tnn, notnull_tmin_map, valid_days=0.0, valid_years=0.0
# )
# tnn_99_00 = berkeley.require_valid(
#     tnn, notnull_tmin_map, valid_days=0.99, valid_years=0.0
# )
# tnn_99_90 = berkeley.require_valid(
#     tnn, notnull_tmin_map, valid_days=0.99, valid_years=0.9
# )

# tnn_00_00_land = utils.land_mean(tnn_00_00, land_mask)
# tnn_99_00_land = utils.land_mean(tnn_99_00, land_mask)
# tnn_99_90_land = utils.land_mean(tnn_99_90, land_mask)

In [ ]:
# tnn_00_00_land.plot(label="0% of days; 0% of years")
# tnn_99_00_land.plot(label="99% of days; 0% of years")
# tnn_99_90_land.plot(label="99% of days; 90% of years")

# plt.legend()

In [ ]:
# utils.calc_anomaly(tnn_00_00_land, CLIMATOLOGY_PERIOD).plot(
#     label="0% of days; 0% of years"
# )
# utils.calc_anomaly(tnn_99_00_land, CLIMATOLOGY_PERIOD).plot(
#     label="99% of days; 0% of years"
# )
# utils.calc_anomaly(tnn_99_90_land, CLIMATOLOGY_PERIOD).plot(
#     label="99% of days; 90% of years"
# )

# plt.legend()

> Restricting the data removes regions in the Tropics, which makes the land mean warmer. Not sure why this has such a strong effect on TNn but not on TXx.

> * TXx has a more "normal" distribution and the removed temperatures are on both sides of the mean
> * TNn has a "U" shaped distribution and the removed temperatures are only above the mean

In [ ]:
import mplotutils as mpu

cmap = copy.copy(plt.get_cmap("RdBu"))
cmap.set_bad("#6a3d9a")

opt = dict(
    add_colorbar=False,
    vmax=0.25,
    extend="both",
    center=0,
    transform=ccrs.PlateCarree(),
    cmap=cmap,
    robust=True,
)


f, axs = plt.subplots(2, 1, sharex=True, subplot_kw=dict(projection=ccrs.PlateCarree()))
axs = axs.flatten()

ax = axs[0]

d = txx_00_00.mean("year") - txx_99_00.mean("year")
h = d.plot(ax=ax, **opt)

ax = axs[1]

d = txx_99_00.mean("year") - txx_99_90.mean("year")
h = d.plot(ax=ax, **opt)

d_txx = d

# ax = axs[2]

# txx_99_90.mean("year").plot(ax=ax, **opt)

for ax in axs:
    ax.set_aspect("equal")
    ax.coastlines()
    ax.add_feature(cfeature.OCEAN, zorder=2, facecolor="white")


print(txx_00_00.mean().item())
print(txx_99_00.mean().item())
print(txx_99_90.mean().item())

mpu.colorbar(h, axs[1], orientation="horizontal")

mpu.set_map_layout(axs)

In [ ]:
# import mplotutils as mpu

# cmap = copy.copy(plt.get_cmap("RdBu"))
# cmap.set_bad("#6a3d9a")

# opt = dict(
#     add_colorbar=False,
#     vmax=0.25,
#     extend="both",
#     center=0,
#     transform=ccrs.PlateCarree(),
#     cmap=cmap,
#     robust=True,
# )


# f, axs = plt.subplots(2, 1, sharex=True, subplot_kw=dict(projection=ccrs.PlateCarree()))
# axs = axs.flatten()

# ax = axs[0]

# d = tnn_00_00.mean("year") - tnn_99_00.mean("year")
# h = d.plot(ax=ax, **opt)

# ax = axs[1]

# d = tnn_99_00.mean("year") - tnn_99_90.mean("year")
# h = d.plot(ax=ax, **opt)

# d_tnn = d


# # ax = axs[2]

# # tnn_99_90.mean("year").plot(ax=ax, **opt)

# for ax in axs:
#     ax.set_aspect("equal")
#     ax.coastlines()
#     ax.add_feature(cfeature.OCEAN, zorder=2, facecolor="white")


# print(tnn_00_00.mean().item())
# print(tnn_99_00.mean().item())
# print(tnn_99_90.mean().item())

# mpu.colorbar(h, axs[1], orientation="horizontal")

# mpu.set_map_layout(axs)

In [ ]:
# lm = d_tnn.isnull() & (land_mask == 1)

# utils.land_mean(tnn_00_00, lm).plot()
# utils.land_mean(tnn_99_00, lm).plot()
# utils.land_mean(tnn_99_90, lm).plot()

# lm.sum() / land_mask.sum() * 100

In [ ]:
lm = d_txx.isnull() & (land_mask == 1)

utils.land_mean(txx_00_00, lm).plot()
utils.land_mean(txx_99_00, lm).plot()

## Comparing data availability TXx vs TNn




In [ ]:
# def h(d):$

# -68..27

bins = np.arange(-6, 52)


# d = d.values[d.notnull()]

year = 1971

plt.hist(txx_00_00.sel(year=year).values.flatten(), alpha=0.5, bins=bins)
plt.hist(txx_99_90.sel(year=year).values.flatten(), alpha=0.5, bins=bins)

plt.axvline(txx_00_00_land.sel(year=year), lw=0.5)
plt.axvline(txx_99_90_land.sel(year=year), lw=0.5)


tnn_00_00_land.sel(year=year)

In [ ]:
# def h(d):$

# -68..27

bins = np.arange(-68, 28)


# d = d.values[d.notnull()]

year = 2021

plt.hist(tnn_00_00.sel(year=year).values.flatten(), alpha=0.5, bins=bins)
plt.hist(tnn_99_90.sel(year=year).values.flatten(), alpha=0.5, bins=bins)

plt.axvline(tnn_00_00_land.sel(year=year), lw=0.5)
plt.axvline(tnn_99_90_land.sel(year=year), lw=0.5)


tnn_00_00_land.sel(year=year)

In [ ]:
cmap = copy.copy(plt.get_cmap("RdBu"))

cmap.set_bad("#6a3d9a")


opt = dict(
    add_colorbar=False,
    #     vmax=0.25,
    #     extend="both",
    #     center=0,
    transform=ccrs.PlateCarree(),
    #     cmap=cmap,
    robust=True,
)

f, axs = plt.subplots(2, 1, sharex=True, subplot_kw=dict(projection=ccrs.Mollweide()))
ax = axs[0]

h = tnn_00_00.sel(year=year).plot(ax=ax, **opt)
ax.coastlines()
mpu.colorbar(h, ax)


ax = axs[1]

# d = tnn_99_00.sel(year=year) - tnn_00_00.sel(year=year)

d = tnn_00_00.sel(year=year)

# d = (d > -30) & (d < 0)

# d = 1. * d

# h = d.plot(ax=ax, cmap=cmap, **opt)

levels = [-60, -25, 0, 20]

h = d.plot.contourf(ax=ax, levels=levels, **opt)


ax.add_feature(cfeature.OCEAN, zorder=2, facecolor="white")

ax.coastlines()
mpu.colorbar(h, ax)